# Analysis based on 1D profiles

In [ ]:
%pylab widget
#%pylab ipympl
#%pylab inline
from plotting_settings import *

import os
cwd = os.getcwd()
plot_dir = os.path.join(cwd, 'plots')
if not os.path.exists(plot_dir):
    os.mkdir(plot_dir)

import sys
sys.path.insert(1, os.path.dirname(cwd))
from analysis import CCUnits

from ppmpy import ppm

from scipy.integrate import cumtrapz
from scipy.signal import convolve

import logging
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

data_path = '/data/CoCo/'

In [ ]:
def cdiff(x):
    dx = 0.5*(np.roll(x, -1) - np.roll(x, +1))
    dx[0] = dx[1] - ((x[2] - x[1]) - (x[1] - x[0]))
    dx[-1] = dx[-2] + (((x[-1] - x[-2])) - (x[-2] - x[-3]))
    return dx

def smooth(arr, kernel):
    na = len(arr)
    nk = len(kernel)
    arr_padded = np.concatenate((np.mean(arr[:nk//2])*np.ones(nk),
                                 arr,
                                 np.mean(arr[-nk//2:])*np.ones(nk)))
    arr_padded = convolve(arr_padded, kernel, mode='same')
    
    return arr_padded[nk:-nk]

def time_averaged_profile(rps, dumps, quantity):
    rp = rps.get_dump(1)
    y = rp.get('Y')
    ndumps = len(dumps)
    q = np.zeros((ndumps, len(y)))
    for i, dmp in enumerate(dumps):
        rp = rps.get_dump(dmp)
        if quantity == 'STDEV_VXZ':
            q[i, :] = (rp.get('STDEV_VX')**2 + \
                       rp.get('STDEV_VZ')**2)**0.5
        elif quantity == 'dX1byX1':
            q[i, :] = rp.get('STDEV_X1')/rp.get('X1')
        elif quantity == 'dAbyA':
            q[i, :] = rp.get('STDEV_A')/rp.get('A')
        else:
            q[i, :] = rp.get(quantity)
    
    avg = np.mean(q, axis=0)
    stdev = np.mean((q - np.tile(avg, (ndumps, 1)))**2, axis=0)**0.5

    return y, avg, stdev

In [ ]:
# Horizontal dimensions of the domain are (2, 2).
horizontal_area = 4.

# The end of the initial transient.
time0 = 500.

# Convective turnover time scale.
tau_conv = 80.
tau_conv_dumps = int(np.round(tau_conv/5.))

rprof_sets = {}
run_lbls = {}
run_lists = {}

codes = ('FLASH', 'MUSIC', 'PPMSTAR', 'PROMPI', 'SLH')

for res in (128, 256, 512):
    for code in codes:
        # We do not have 512^3 runs from all codes.
        if (res == 512) and (code not in ('PPMSTAR', 'PROMPI')):
            continue
        
        rid = code+'-'+str(res)
        run_path = os.path.join(data_path, '1D-profiles', code, rid)
        rprof_sets[rid] = ppm.RprofSet(run_path, geometry='cartesian')
        run_lbls[rid] = r'{:s}, ${:d}^3$'.format(code, res)
        print('')
    
    run_lists[res] = [rid for rid in run_lbls.keys() if str(res) in rid]

## Initial stratification

In [ ]:
res = 256
runs = np.append(run_lists[res], run_lists[512])

for i, q in enumerate(('P', 'RHO', 'TEMP', 'A')):
    ifig=i+1; plt.close(ifig)
    figure(ifig, figsize=(width, 0.75*width), dpi=scrdpi)
    for j, rid in enumerate(runs):
        dmp0 = 0
        # PPMSTAR does not output the initial state.
        if rid.split('-')[0] in ('PPMSTAR'):
            dmp0 = 1
        rp = rprof_sets[rid].get_dump(dmp0)
        semilogy(rp.get('Y'), rp.get(q), ls=ls[j], color=lc[j], label=run_lbls[rid])
    legend(loc=0)
    xlabel('Y')
    ylabel(q)
    tight_layout()

In [ ]:
rp = rprof_sets['SLH-256'].get_dump(0)
y = rp.get('Y')
p = rp.get('P')
rho = rp.get('RHO')
A = rp.get('A')
X1 = rp.get('X1')

Hp = -cdiff(y)/cdiff(np.log(p))
grav = p/(rho*Hp)
nabla_ad = 0.4
nabla_rho = cdiff(np.log(rho))/cdiff(np.log(p))
BVF = ((grav/Hp)*(nabla_ad - 1 + nabla_rho))**0.5

mu0 = 1.848
mu1 = 1.802
mu = mu0*(1. - X1) + mu1*X1

mm = np.load('mesa_model_29350.npy', allow_pickle=True).item()
ccu = CCUnits()
mm_r = mm['r']/ccu.length
mm_p = mm['pres']/ccu.pressure
mm_rho = mm['rho']/ccu.density
mm_A = mm_p/mm_rho**(5./3.)
mm_mu = mm['mu']

ifig=5; close(ifig);
figure(ifig, figsize=(width, 0.65*width), dpi=scrdpi)
lw = 1.5
mm_lw = 0.75

ax1 = gca()

ax1.semilogy(mm_r, mm_p, ls=ls[0], lw=mm_lw, color=lc[0])
ln1 = ax1.semilogy(y, p, ls=ls[0], lw=lw, color=lc[0], label=r'$p$')

ax1.semilogy(mm_r, mm_rho, ls=ls[1], lw=mm_lw, color=lc[1])
ln2 = ax1.semilogy(y, rho, ls=ls[1], lw=lw, color=lc[1], label=r'$\rho$')

ax1.semilogy(mm_r, mm_A, ls=ls[2], lw=mm_lw, color=lc[2])
ln3 = ax1.semilogy(y, A, ls=ls[2], lw=lw, color=lc[2], label=r'$A$')
ax1.set_xlim((1., 3.))
ax1.set_ylim((6e-3, 5e0))
ax1.set_xlabel(r'$y$')
ax1.set_ylabel(r'$p,\,\rho,\,A$')

ax2 = ax1.twinx()
ax2.plot(mm_r, mm_mu, ls=ls[3], lw=mm_lw, color=lc[3])
ln4 = ax2.plot(y, mu, ls=ls[3], lw=lw, color=lc[3], label=r'$\mu$')
ax2.set_xlim((1., 3.))
ax2.set_ylim((1.7975, 1.85))
ax1.set_xlabel(r'$r$, $y$')
ax2.set_ylabel(r'$\mu$')

lns = ln1 + ln2 + ln3 + ln4
lbls = [l.get_label() for l in lns]
ax2.legend(lns, lbls, loc='lower left')

tight_layout()
savefig(os.path.join(plot_dir, 'stratification.pdf'), dpi=savedpi)

ifig=6; close(ifig);
figure(ifig, figsize=(width, 0.65*width), dpi=scrdpi)
plot(y, BVF)
N0 = 0.42
axhline(y=N0, ls=':', color='k', label='N = {:.2f}'.format(N0))
legend(loc='lower right')
xlim((1., 3.))
xlabel(r'$y$')
ylabel(r'Brunt–Väisälä frequency $N$')
tight_layout()

## Precompute some quantities

In [ ]:
time = {}
yub = {}
hub = {}
me = {}
rhoub = {}
v_rms_cl = {}
v_rms_sl = {}
A_lcz = {}
cd = {}
for i, rid in enumerate(run_lbls.keys()):
    rps = rprof_sets[rid]
    dumps = array(rps.get_dump_list())
    time[rid] = np.array([rprof_sets[rid].get_dump(dmp).get('t') for dmp in dumps])
    
    # Impose a time limit to make sure that all runs
    # are treated the same way if smoothing is used.
    sl = (time[rid] < 2003.)
    dumps = dumps[sl]
    time[rid] = time[rid][sl]
    print('{:s}, {:d} dumps, t_end = {:.1f}'.format(rid, len(dumps), time[rid][-1]))
    
    yub[rid], hub[rid] = rps.bound_rad(dumps, 0., 3., var='X1', criterion='max_grad', \
                                       return_var_scale_height=True)
    _, me[rid], _ = rps.entr_rate(dumps, 0., 3., var='X1', criterion='max_grad', 
                                  show_plots=False, return_time_series=True)
    
    rhoub[rid] = np.zeros(len(dumps))
    v_rms_cl[rid] = np.zeros(len(dumps))
    v_rms_sl[rid] = np.zeros(len(dumps))
    A_lcz[rid] = np.zeros(len(dumps))
    cd[rid] = np.zeros(len(dumps))
    rp = rps.get_dump(dumps[0])
    y = rp.get('Y')
    dy = -cdiff(y)
    dV = horizontal_area*dy
    for j, dmp in enumerate(dumps):
        rp = rps.get_dump(dmp)
        rho = rp.get('RHO')
        x1 = rp.get('X1')
        A = rp.get('A')
        fh = rp.get('FH')
        fk = rp.get('FK')
        
        idx0 = np.argmin(np.abs(y - yub[rid][j]))
        rhoub[rid][j] = rho[idx0]
        
        v2 = rp.get('STDEV_VX')**2 + \
             rp.get('STDEV_VY')**2 + \
             rp.get('STDEV_VZ')**2
        dm = rho*dV
        
        # Convective layer.
        idx_min = np.argmax(y < yub[rid][j] - 0.1)
        v_rms_cl[rid][j] = (np.sum((v2*dm)[idx_min:])/np.sum(dm[idx_min:]))**0.5

        # Stable layer.
        idx_max = np.argmin(y > yub[rid][j] + 0.1)
        v_rms_sl[rid][j] = (np.sum((v2*dm)[:idx_max])/np.sum(dm[:idx_max]))**0.5
        
        y_lcz = 1. + (2./3.)*(yub[rid][j] - 1.)
        idx_lcz = np.argmin(np.abs(y - y_lcz))
        A_lcz[rid][j] = np.mean(A[idx_lcz:])
        
        idx0 = np.argmin(np.abs(y - 1.5))
        cd[rid][j] = fh[idx0]    
    print('')

## Velocity field

In [ ]:
res = 256
runs = np.append(run_lists[res], ['PPMSTAR-512', 'PROMPI-512'])

ifig=7; close(ifig);
figure(ifig, figsize=(width, 0.7*width), dpi=scrdpi)
for i, rid in enumerate(runs):
    plot(time[rid], v_rms_cl[rid], ls=ls[i], color=lc[i], label=run_lbls[rid])
    plot(time[rid], v_rms_sl[rid], ls=ls[i], color=lc[i])
    
    idx0 = np.argmin(np.abs(time[rid] - time0))
    mean_cl = np.mean(v_rms_cl[rid][idx0:])
    sigma_cl = np.mean((v_rms_cl[rid][idx0:] - mean_cl)**2)**0.5
    p = np.poly1d(np.polyfit(time[rid][idx0:], v_rms_cl[rid][idx0:], 1))
    #plot(time[rid], p(time[rid]), ls=ls[i], color=lc[i])
    
    change_cl = (p(time[rid][-1]) - p(time[rid][idx0]))
    mean_sl = np.mean(v_rms_sl[rid][idx0:])
    print('{:s}, {:.4f}, {:.1f}, {:.1f}, {:.4f}'.format(rid, mean_cl, \
          100.*3.*sigma_cl/mean_cl, 100.*change_cl/mean_cl,  mean_sl))
axvline(x=time0, ls=':', color='k')
legend(loc='center right', ncol=2)
text(550, 0.0395, 'convective layer')
text(550, 0.005, 'stable layer')
xlim((0., 2e3))
ylim((0., 0.0425))
xlabel(r'$t$')
ylabel(r'$\widetilde{v}_\mathrm{rms}$')
tight_layout()
savefig(os.path.join(plot_dir, 'v_rms_evolution_{:d}.pdf'.format(res)), dpi=savedpi)

In [ ]:
q = 'v_rms_cl'
#q = 'FH'

ylims = {'v_rms_cl':(-0.5, 1.2),
         'FH':(-0.7, 1.1)}
ylbls = {'v_rms_cl':r'$R(\Delta t)$',
         'FH':r'$R_{\mathcal{F}_H}(\Delta t)$'}

ifig=8; close(ifig);
figure(ifig, figsize=(width, 0.7*width), dpi=scrdpi)
for k, runs in enumerate([run_lists[128], run_lists[256], ['PPMSTAR-512']]):
    for i, rid in enumerate(runs):
        idx0 = np.argmin(np.abs(time[rid] - time0))

        # Remove the initial transient.
        t = np.copy(time[rid][idx0:])
        if q == 'v_rms_cl':
            dd = np.copy(v_rms_cl[rid][idx0:])
        elif q == 'FH':
            dd = np.copy(cd[rid][idx0:])

        # Remove any linear trend.
        coeffs = np.polyfit(t, dd, 1)
        dd_fit = coeffs[0]*t + coeffs[1]
        dd -= dd_fit

        # Make the time series periodic.
        dd = np.append(dd, dd[::-1])

        # Compute the auto-correlation function.
        cf = np.zeros_like(t)
        for j, shift in enumerate(range(len(cf))):
            cf[j] = np.sum(dd*np.roll(dd, shift))
        cf /= np.max(cf)
        t_shift = t - t[0]

        col = lc[i] if '512' not in rid else lc[5]
        semilogx(t_shift/tau_conv, cf, ls=ls[i], color=col, \
                 lw=0.6+0.4*k, label=run_lbls[rid])
axhline(y=0., ls=':', color='k')
legend(loc='upper right', ncol=2, fontsize=5.5)
xlim((1e-1, 2e1))
ylim(ylims[q])
xlabel(r'$\Delta t / \tau_\mathrm{conv}$')
ylabel(ylbls[q])
tight_layout()
savefig(os.path.join(plot_dir, '{:s}_correlation_time_scale.pdf'.format(q, res)), dpi=savedpi)

In [ ]:
res = 256
runs_band = run_lists[res]
runs = np.append(runs_band, ['PPMSTAR-512'])

# 6 turnovers centred aroud t = 1250
dumps = range(202, 299)

# 6 turnovers centred aroud t = 1000
#dumps = range(152, 249)

# 3 turnovers centred aroud t = 1250
#dumps = range(226, 275)

nconv = (dumps[-1] - dumps[0])/tau_conv_dumps
print('nconv = {:.2f}'.format(nconv))

ifig=9; close(ifig);
figure(ifig, figsize=(width, 0.7*width), dpi=scrdpi)
ax0 = gca()
ax1 = ax0.twinx()
rp = rprof_sets[runs_band[0]].get_dump(dumps[0])
y_band = rp.get('Y')
avg_vy_band = np.zeros_like(y_band)
stdev_vy_band = np.zeros_like(y_band)
avg_vxz_band = np.zeros_like(y_band)
stdev_vxz_band = np.zeros_like(y_band)
for j, rid in enumerate(runs):
    rp = rprof_sets[rid].get_dump(dumps[0])
    y = rp.get('Y')
    
    vy_rms = np.zeros_like(y)
    vxz_rms = np.zeros_like(y)
    y, avg_vy, stdev_vy = time_averaged_profile(rprof_sets[rid], dumps, 'STDEV_VY')
    y, avg_vxz, stdev_vxz = time_averaged_profile(rprof_sets[rid], dumps, 'STDEV_VXZ')
    
    ax0.plot(y, avg_vy, ls=ls[j], color=lc[j], lw=4./3., label=run_lbls[rid])
    ax1.plot(y, avg_vxz, ls=ls[j], color=lc[j], lw=3./4.)
    
    if rid in runs_band:
        avg_vy_band += avg_vy
        stdev_vy_band += stdev_vy
        avg_vxz_band += avg_vxz
        stdev_vxz_band += stdev_vxz

avg_vy_band /= len(runs_band)
stdev_vy_band /= len(runs_band)
stdev_vy_band /= nconv**0.5
avg_vxz_band /= len(runs_band)
stdev_vxz_band /= len(runs_band)
stdev_vxz_band /= nconv**0.5
ax1.fill_between(y_band, avg_vxz_band - 3*stdev_vxz_band, \
                 avg_vxz_band + 3*stdev_vxz_band, \
                 facecolor='0.8')
ax0.fill_between(y_band, avg_vy_band - 3*stdev_vy_band, \
                 avg_vy_band + 3*stdev_vy_band, \
                 facecolor='0.8')

ax0.legend(loc='upper center', ncol=2)
ax0.set_xlabel('$y$')
ax0.set_ylabel(r'$\langle \widetilde{v}_{y,\mathrm{rms}} \rangle$')
ax1.set_ylabel(r'$\langle \widetilde{v}_\mathrm{xz,rms} \rangle$')
ax0.set_xlim((1., 3.))
ax0.set_ylim((0., 0.06))
ax1.set_xlim((1., 3.))
ax1.set_ylim((0., 0.04))
ax1.set_yticks(0.01*np.arange(5))
tight_layout()
savefig(os.path.join(plot_dir, 'v_rms_profiles_{:d}.pdf'.format(res)), dpi=savedpi)

## Convective boundary and mass entrainment

In [ ]:
res = 256
runs = np.append(run_lists[res], ['PPMSTAR-512'])
dump_ranges = [range(100, 132), range(234, 266), range(369, 401)]

ifig=10; close(ifig);
figure(ifig, figsize=(width, 0.7*width), dpi=scrdpi)

rp = rprof_sets['SLH-256'].get_dump(0)
y = rp.get('Y')
x1 = rp.get('X1')
plot(y, x1, ls=':', color='k', label=r'$t = 0$')

dy = -cdiff(y)
dm = dy
rho = rp.get('RHO')
idx_min = np.argmax(x1 < 1.)
m_total = np.sum((rho*dm)[idx_min:])
m1_total = np.sum(x1*rho*dm)
m1_transition = np.sum((x1*rho*dm)[idx_min:])
print('At t = 0:')
print('X1 transition ends at y = {:.4f}.'.format(y[idx_min]))
print('Total mass under the transition per unit area: {:.3e}.'.format(m_total))
print('Total mu1 fluid mass per unit area: {:.3e}.'.format(m1_total))
print('Fluid mu1 fluid mass per unit area in the transition layer: {:.3e}.'.format(m1_transition))

for j, rid in enumerate(runs):
    for k, dumps in enumerate(dump_ranges):
        y, x1, _ = time_averaged_profile(rprof_sets[rid], dumps, 'X1')
        lbl = run_lbls[rid] if k == 0 else ''
        plot(y, x1, ls=ls[j], color=lc[j], label=lbl)
legend(loc='upper left')
xlabel('$y$')
ylabel(r'$\langle \widetilde{X}_1 \rangle$')
xlim((1.6, 2.7))
ylim((-0.05, 1.05))
tight_layout()
savefig(os.path.join(plot_dir, 'X1_profiles_{:d}.pdf'.format(res)), dpi=savedpi)

In [ ]:
res = 256
runs = append(run_lists[res], ['PPMSTAR-512', 'PROMPI-512'])

ifig=11; close(ifig);
figure(ifig, figsize=(width, 0.7*width), dpi=scrdpi)
for i, rid in enumerate(runs):
    plot(time[rid], yub[rid], ls=ls[i], color=lc[i], label=run_lbls[rid])
    
    idx0 = np.argmin(np.abs(time[rid] - time0))
    yub_avg = np.mean(yub[rid][idx0:])
    print('{:s}, yub_avg = {:.3f}'.format(rid, yub_avg))
axvline(x=time0, ls=':', color='k')
legend(loc='lower right', ncol=2)
xlim((0., 2e3))
ylim((2., 2.55))
xlabel(r'$t$')
ylabel(r'$y_\mathrm{ub}$')
tight_layout()
savefig(os.path.join(plot_dir, 'yub_evolution_{:d}.pdf'.format(res)), dpi=savedpi)

ifig=12; close(ifig);
figure(ifig, figsize=(width, 0.7*width), dpi=scrdpi)
for i, rid in enumerate(runs):
    kw = 16
    # PROMPI-512 has a higher output frequency.
    if rid == 'PROMPI-512':
        kw *= 4
    kernel = np.ones(kw)/float(kw)
    hub_smoothed = smooth(hub[rid], kernel)
    plot(time[rid], hub_smoothed, ls=ls[i], color=lc[i], label=run_lbls[rid])
axvline(x=time0, ls=':', color='k')
axhline(y=0.0404, ls=':', color='k')
legend(loc='lower right', ncol=2)
xlim((0., 2e3))
ylim((0., 0.075))
xlabel(r'$t$')
ylabel(r'$H_\mathrm{ub}$')
tight_layout()
savefig(os.path.join(plot_dir, 'hub_evolution_{:d}.pdf'.format(res)), dpi=savedpi)

In [ ]:
res = 256
runs = append(run_lists[res], ['PPMSTAR-512', 'PROMPI-512'])

ifig=13; close(ifig);
figure(ifig, figsize=(width, 0.7*width), dpi=scrdpi)
for i, rid in enumerate(runs):
    plot(time[rid], me[rid], ls=ls[i], color=lc[i], label=run_lbls[rid])
plot([0., time0], [1.665e-02]*2, ls=':', color='k')
axvline(x=time0, ls=':', color='k')
legend(loc='lower right', ncol=2)
xlim((0., 2e3))
ylim((0., 0.09))
xlabel(r'$t$')
ylabel(r'$M_\mathrm{e}$')
tight_layout()
savefig(os.path.join(plot_dir, 'me_evolution_{:d}.pdf'.format(res)), dpi=savedpi)

ifig=14; close(ifig);
figure(ifig, figsize=(width, 0.7*width), dpi=scrdpi)
for i, rid in enumerate(runs):
    dmedt = cdiff(me[rid])/cdiff(time[rid])
    kw = 48
    # PROMPI-512 has a higher output frequency.
    if rid == 'PROMPI-512':
        kw *= 4
    kernel = np.ones(kw)/float(kw)
    dmedt_smoothed = smooth(dmedt, kernel)
    semilogy(time[rid], dmedt_smoothed, ls=ls[i], color=lc[i], label=run_lbls[rid])
axvline(x=time0, ls=':', color='k')
legend(loc='upper right', ncol=2)
xlim((0., 2e3))
ylim((1e-5, 1.5e-4))
xlabel(r'$t$')
ylabel(r'$\dot{M}_\mathrm{e}$')
tight_layout()
savefig(os.path.join(plot_dir, 'dmedt_evolution_{:d}.pdf'.format(res)), dpi=savedpi)

In [ ]:
res = 256
runs = append(run_lists[res], ['PPMSTAR-512', 'PROMPI-512'])

ifig=15; close(ifig);
figure(ifig, figsize=(width, 0.7*width), dpi=scrdpi)
for i, rid in enumerate(runs):
    plot(time[rid], rhoub[rid], ls=ls[i], color=lc[i], label=run_lbls[rid])
legend(loc='upper right', ncol=2)
xlabel(r'$t$')
ylabel(r'$\rho_\mathrm{ub}$')
tight_layout()

In [ ]:
res = 256
runs = append(run_lists[res], ['PPMSTAR-512', 'PROMPI-512'])

ifig=16; close(ifig);
figure(ifig, figsize=(width, 0.7*width), dpi=scrdpi)
mean_dAdt = 0.
for i, rid in enumerate(runs):
    t = time[rid]
    A = A_lcz[rid]
    dAdt = cdiff(A)/cdiff(t)
    idx0 = np.argmin(np.abs(t - time0))
    mean_dAdt += np.mean(dAdt[idx0:])
    plot(t, dAdt, ls=ls[i], color=lc[i], label=run_lbls[rid])
#     kw = 16
#     kernel = np.ones(kw)/float(kw)
#     dAdt_smooth = smooth(dAdt, kernel)
#     plot(t, dAdt_smooth, ls=ls[i], color=lc[i], label=run_lbls[rid])
xlabel(r'$t$')
ylabel(r'$\mathrm{d}A / \mathrm{d}t$')
legend(loc='lower right', ncol=2)
mean_dAdt /= len(runs)
axhline(y=mean_dAdt, ls=':', color='k')
axvline(x=time0, ls=':', color='k')
xlim((0., 2e3))
tight_layout()
print('mean_dAdt = ', mean_dAdt)

rp = rprof_sets['SLH-256'].get_dump(0)
y = rp.get('Y')
rho = rp.get('RHO')
X1 = rp.get('X1')
A = rp.get('A')

m1 = np.cumsum(X1*rho*cdiff(y))
m1 -= m1[-1]

dAdm1 = cdiff(A)/cdiff(m1)

ifig=17; close(ifig);
figure(ifig, figsize=(width, 0.7*width), dpi=scrdpi)
plot(m1, dAdm1)
xlim((0., 0.14))
ylim((0., 14.))
xlabel(r'$m_1$')
ylabel(r'$\mathrm{d}A / \mathrm{d}m_1$')
tight_layout()

ifig=18; close(ifig);
figure(ifig, figsize=(width, 0.7*width), dpi=scrdpi)
me_heating = m1
dmedt_heating = mean_dAdt/dAdm1
semilogy(me_heating, dmedt_heating)
xlim((0., 0.14))
xlabel(r'$M_\mathrm{e,heating}$')
ylabel(r'$\dot{M}_\mathrm{e,heating}$')
tight_layout()

## Fluctuations and transport of energy

In [ ]:
res = 256
runs_band = run_lists[res]
runs = np.append(runs_band, ['PPMSTAR-512'])

# 6 turnovers centred aroud t = 1250
dumps = range(202, 299)

nconv = (dumps[-1] - dumps[0])/tau_conv_dumps
print('nconv = {:.2f}'.format(nconv))

ifig=19; close(ifig);
figure(ifig, figsize=(width, 0.7*width), dpi=scrdpi)
rp = rprof_sets[runs_band[0]].get_dump(dumps[0])
y_band = rp.get('Y')
avg_dX1byX1_band = np.zeros_like(y_band)
stdev_dX1byX1_band = np.zeros_like(y_band)    
avg_dAbyA_band = np.zeros_like(y_band)
stdev_dAbyA_band = np.zeros_like(y_band)    
for j, rid in enumerate(runs):
    y, avg_dX1byX1, stdev_dX1byX1 = time_averaged_profile(rprof_sets[rid], dumps, 'dX1byX1')
    y, avg_dAbyA, stdev_dAbyA = time_averaged_profile(rprof_sets[rid], dumps, 'dAbyA')
    
    semilogy(y, avg_dAbyA, ls=ls[j], color=lc[j], lw=3./4., label=run_lbls[rid])
    semilogy(y, avg_dX1byX1, ls=ls[j], lw=4./3., color=lc[j])
    if rid in runs_band:
        avg_dX1byX1_band += avg_dX1byX1
        stdev_dX1byX1_band += stdev_dX1byX1
        avg_dAbyA_band += avg_dAbyA
        stdev_dAbyA_band += stdev_dAbyA
avg_dX1byX1_band /= len(runs_band)
stdev_dX1byX1_band /= len(runs_band)
stdev_dX1byX1_band /= nconv**0.5
avg_dAbyA_band /= len(runs_band)
stdev_dAbyA_band /= len(runs_band)
stdev_dAbyA_band /= nconv**0.5
fill_between(y_band, avg_dX1byX1_band - 3*stdev_dX1byX1_band, \
             avg_dX1byX1_band + 3*stdev_dX1byX1_band, \
             facecolor='0.8')
fill_between(y_band, avg_dAbyA_band - 3*stdev_dAbyA_band, \
             avg_dAbyA_band + 3*stdev_dAbyA_band, \
             facecolor='0.8')
legend(loc=0)
xlabel(r'$y$')
ylabel(r'$\langle \Delta \overline{A}\,/\,\overline{A} \rangle$, '
       r'$\langle \Delta \widetilde{X}_1\,/\,\widetilde{X}_1 \rangle$')
xlim((1., 3.))
ylim((8e-4, 6e0))
legend(loc='upper left')
tight_layout()
savefig(os.path.join(plot_dir, 'fluctuations_{:d}.pdf'.format(res)), dpi=savedpi)

In [ ]:
res = 256
runs_band = run_lists[res]
runs = np.append(runs_band, ['PPMSTAR-512'])

dumps = range(100, 401)
nconv = (dumps[-1] - dumps[0])/tau_conv_dumps
print('nconv = {:.2f}'.format(nconv))

qlbl = {'FH':r'$\langle \mathcal{F}_H \rangle$',
        'FK':r'$\langle \mathcal{F}_k \rangle$',
        'FFD':r'$\langle f_\mathrm{d} \rangle$'}

qlim = {'FH':(-1.5e-6, 2.7e-5),
        'FK':(-8e-7, 1.4e-6),
        'FFD':(0.44, 0.57)}

lloc = {'FH':'upper right',
        'FK':'upper right',
        'FFD':'lower center'}

ifig = 20
for quantity in ('FH', 'FK', 'FFD'):
    close(ifig)
    figure(ifig, figsize=(width, 0.7*width), dpi=scrdpi)
    ifig += 1
    rp = rprof_sets[runs_band[0]].get_dump(dumps[0])
    y_band = rp.get('Y')
    avg_band = np.zeros_like(y_band)
    stdev_band = np.zeros_like(y_band)
    for i, rid in enumerate(runs):
        y, avg, stdev = time_averaged_profile(rprof_sets[rid], dumps, quantity)
        q0 = 0.5 if quantity == 'FFD' else 0.
        axhline(y=q0, ls=':', color='k')
        plot(y, avg, ls=ls[i], color=lc[i], label=run_lbls[rid])
        
        if rid in runs_band:
            avg_band += avg
            stdev_band += stdev
    avg_band /= len(runs_band)
    stdev_band /= len(runs_band)
    stdev_band /= nconv**0.5
    fill_between(y_band, avg_band - 3*stdev_band, avg_band + 3*stdev_band, \
                 facecolor='0.8')

    legend(loc=lloc[quantity], ncol=2)
    xlabel('$y$')
    ylabel(qlbl[quantity])
    xlim((1., 3.))
    ylim(qlim[quantity])
    if quantity in ('FH', 'FK'):
        ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    tight_layout()
    savefig(os.path.join(plot_dir, '{:s}_profiles_{:d}.pdf'.format(quantity, res)), dpi=savedpi)
    
    if quantity == 'FK':
        close(ifig)
        figure(ifig, figsize=(width, 0.7*width), dpi=scrdpi)
        ifig += 1
        rid = 'PPMSTAR-512'

        y, fk, _ = time_averaged_profile(rprof_sets[rid], dumps, 'FK')
        y, ffd, _ = time_averaged_profile(rprof_sets[rid], dumps, 'FFD')
        y, rho, _ = time_averaged_profile(rprof_sets[rid], dumps, 'RHO')
        y, stdev_vx, _ = time_averaged_profile(rprof_sets[rid], dumps, 'STDEV_VX')
        y, stdev_vy, _ = time_averaged_profile(rprof_sets[rid], dumps, 'STDEV_VY')
        y, stdev_vz, _ = time_averaged_profile(rprof_sets[rid], dumps, 'STDEV_VZ')
        stdev_v = (stdev_vx**2 + stdev_vy**2 + stdev_vz**2)**0.5

        geom_fac = ffd*((1. - ffd)/ffd)**(3./2.)*((ffd/(1 - ffd))**2 - 1.)
        fk_ts = 0.5*rho*stdev_v**2*stdev_vy*geom_fac

        axhline(y=0., ls=':', color='k')
        plot(y, fk, ls=ls[5], color=lc[5], label=run_lbls[rid])
        plot(y, fk_ts, ls=ls[1], color=lc[1], label='two-stream model')
        fill_between(y_band, avg_band - 3*stdev_band, avg_band + 3*stdev_band, \
                     facecolor='0.8')
        legend(loc='upper right')
        xlabel('$y$')
        ylabel(r'$\langle \mathcal{F}_k \rangle$')
        xlim((1., 3.))
        ylim(qlim[quantity])
        ticklabel_format(style='sci', axis='y', scilimits=(0,0))
        tight_layout()
        savefig(os.path.join(plot_dir, 'FK_profiles_two_stream.pdf'.format(quantity, res)), dpi=savedpi)